# Import

In [1]:
import requests
import tabulate
import future

import pandas as pd
import numpy as np

# pd.read_csv("/kaggle/input/clean-creditcard-train-data/clean-credit-train.csv")
# !pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [2]:
# Required for plotting:
import matplotlib
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

h2o:1: DeprecationWarning: Your Python version is 3.10.10. The support for this version will be removed in H2O 3.42.0.1.


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmt0_4jia
  JVM stdout: /tmp/tmpmt0_4jia/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmt0_4jia/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,1 month and 12 days
H2O_cluster_name:,H2O_from_python_unknownUser_6dr5jc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


## Import Data

In [7]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("/kaggle/input/clean-creditcard-train-data/clean-credit-train.csv")
test = h2o.import_file("/kaggle/input/clean-creditcard-test-data/clean-credit-test.csv")

# Identify predictors and response
x = train.columns
print(x)
y = "Class"
x.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']


## Convert the target to factor (Categorical)

In [8]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

## Train the model

In [10]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=3, seed=1)
aml.train(x=x, y=y, training_frame=train)



AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_2_20230517_63622


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    163                163                         766771                 15           15           15            108           710           369.485

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 5.022656839270833e-09
RMSE: 7.087070508518194e-05
LogLoss: 2.7328193386006475e-06
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999251145067237
       0       1       Error    Rate
-----  ------  ------  -------  --------------
0      284315  0       0        (0.0/284315.0)
1      0       284315  0        (0.0/284315.0)
Total  284315  284315  0        (0.0/568630.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999925     1         69
max f2                       0.999925     1         69
max f0point5                 0.999925     1         69
max accuracy                 0.999925     1         69
max precision                1            1         0
max recall                   0.999925     1         69
max specificity              1            1         0
max absolute_mcc             0.999925     1         69
max min_per_class_accuracy   0.999925     1         69
max mean_per_class_accuracy  0.999925     1         69
max tns                      1            284315    0
max fns                      1            248119    0
max fps                      1.39826e-07  284315    399
max tps                      0.999925     284315    69
max tnr                      1            1         0
max fnr                      1            0.872691  0
max fpr                      1.39826e-07  1         399
max tpr                      0.999925     1         69

Gains/Lift Table: Avg response rate: 50.00 %, avg score: 50.00 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain    cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  ------  -----------------  --------------------
1        0.0103406                   1                  2       2                  1                1            1                           1                   0.0206813       0.0206813                  100     100                0.0206813
2        0.0203735                   1                  2       2                  1                1            1                           1                   0.0200658       0.0407471                  100     100                0.0407471
3        0.030176                    1                  2       2                  1                1            1                           1                   0.019605        0.0603521                  100     100                0.0603521
4        0.0403795                   1                  2       2                  1                1            1                           1                   0.0204069       0.080759                   100     100                0.080759
5        0.050532                    1                  2       2                  1                1            1                           1                   0.0203049       0.101064                

## Leader board

In [11]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_1_AutoML_2_20230517_63622,0.999998,0.000347033,0.999998,1.23103e-05,0.00726994,5.2852e-05
StackedEnsemble_BestOfFamily_1_AutoML_2_20230517_63622,0.999998,0.000362196,0.999997,1.05517e-05,0.00692811,4.79988e-05
XGBoost_1_AutoML_2_20230517_63622,0.999994,0.000503743,0.999993,1.58275e-05,0.00940028,8.83653e-05
GLM_1_AutoML_2_20230517_63622,0.9859,0.137569,0.988587,0.0502664,0.198522,0.0394111


## Best Model

In [12]:
aml.leader

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_2_20230517_63622


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    163                163                         766771                 15           15           15            108           710           369.485

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 5.022656839270833e-09
RMSE: 7.087070508518194e-05
LogLoss: 2.7328193386006475e-06
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999251145067237
       0       1       Error    Rate
-----  ------  ------  -------  --------------
0      284315  0       0        (0.0/284315.0)
1      0       284315  0        (0.0/284315.0)
Total  284315  284315  0        (0.0/568630.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999925     1         69
max f2                       0.999925     1         69
max f0point5                 0.999925     1         69
max accuracy                 0.999925     1         69
max precision                1            1         0
max recall                   0.999925     1         69
max specificity              1            1         0
max absolute_mcc             0.999925     1         69
max min_per_class_accuracy   0.999925     1         69
max mean_per_class_accuracy  0.999925     1         69
max tns                      1            284315    0
max fns                      1            248119    0
max fps                      1.39826e-07  284315    399
max tps                      0.999925     284315    69
max tnr                      1            1         0
max fnr                      1            0.872691  0
max fpr                      1.39826e-07  1         399
max tpr                      0.999925     1         69

Gains/Lift Table: Avg response rate: 50.00 %, avg score: 50.00 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain    cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  ------  -----------------  --------------------
1        0.0103406                   1                  2       2                  1                1            1                           1                   0.0206813       0.0206813                  100     100                0.0206813
2        0.0203735                   1                  2       2                  1                1            1                           1                   0.0200658       0.0407471                  100     100                0.0407471
3        0.030176                    1                  2       2                  1                1            1                           1                   0.019605        0.0603521                  100     100                0.0603521
4        0.0403795                   1                  2       2                  1                1            1                           1                   0.0204069       0.080759                   100     100                0.080759
5        0.050532                    1                  2       2                  1                1            1                           1                   0.0203049       0.101064                

## Predict

In [13]:
# To generate predictions on a test set, you can make predictions
# directly on the `H2OAutoML` object or on the leader model
# object directly
# preds = aml.predict(test)

# or on leader model:
preds = aml.leader.predict(test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [14]:
preds

predict,p0,p1
0,1,3.87331e-07
0,0.999999,7.42735e-07
0,0.999995,4.5036e-06
0,0.999999,9.3066e-07
0,1,2.18236e-07
0,0.999999,1.09348e-06
0,0.999999,5.15381e-07
0,1,3.33872e-07
0,0.999999,1.47252e-06
0,1,2.52142e-07


In [16]:
dfpd = preds.as_data_frame()

dfpd.to_csv('prediction-outputs.csv')


In [19]:
# save the model
model_path = h2o.save_model(model=aml.leader, path="/tmp/mymodel", force=True)
print(model_path)




/tmp/mymodel/GBM_1_AutoML_2_20230517_63622


In [25]:
# load the model
saved_model = h2o.load_model(model_path)
print(saved_model)
# download the model built above to your local machine
my_local_model = h2o.download_model(aml.leader)
my_local_model

Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_2_20230517_63622


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    163                163                         766771                 15           15           15            108           710           369.485

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 5.022656839270833e-09
RMSE: 7.087070508518194e-05
LogLoss: 2.7328193386006475e-06
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999251145067237
       0       1       Error    Rate
-----  ------  ------  -------  --------------
0      284315  0       0        (0.0/284315.0)
1    

'/kaggle/working/GBM_1_AutoML_2_20230517_63622'

## Get metrics for leaderboard

In [ ]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb